# LSTM 실습 : 가상화폐(이더리움) 가격 예측

## A. 데이터 탐색

In [0]:
# General packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [0]:
data = pd.read_csv('eth.csv')
data.head()

In [0]:
high_prices = data['High'].values
low_prices = data['Low'].values
mid_prices = (high_prices + low_prices) / 2

## B. 데이터 정규화

In [0]:
def NormalizeWindows(data):
    
    normalized_data = []
    for window in data:
        normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalized_data.append(normalized_window)
    
    return np.array(normalized_data)

In [0]:
seq_len = 50
sequence_length = seq_len + 1

eth_data = []

for index in range(len(mid_prices) - sequence_length):
    eth_data.append(mid_prices[index: index + sequence_length])

eth_data = NormalizeWindows(eth_data)

## C. 데이터 분리

In [0]:
# split train and test data
row = int(round(eth_data.shape[0] * 0.9))
train = eth_data[:row, :]
np.random.shuffle(train)

input_train = train[:, :-1]
input_train = np.reshape(input_train, (input_train.shape[0], input_train.shape[1], 1))
label_train = train[:, -1]

input_test = eth_data[row:, :-1]
input_test = np.reshape(input_test, (input_test.shape[0], input_test.shape[1], 1))
label_test = eth_data[row:, -1]

input_train.shape, input_test.shape

## D. LSTM 신경망 모델 수립

### 신경망 구조 정의

In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

In [0]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))

### 옵티마이저 선택 및 컴파일

In [0]:
from keras import optimizers
Adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = Adam, loss = 'mse')

### 모델 Fitting

In [0]:
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

model.fit(input_train, label_train,
    validation_data=(input_test, label_test),
    batch_size=10,
    epochs=20,
    callbacks=[TensorBoard(log_dir='logs/%s' % (start_time)),
               ModelCheckpoint('./models/%s_eth.h5' % (start_time), 
                                monitor = 'val_loss', 
                            verbose = 1, 
                            save_best_only = True, 
                            mode = 'auto'),
            ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.2, 
                              patience = 5, 
                              verbose = 1, 
                              mode = 'auto')])

## E. 결과 확인

In [0]:
pred = model.predict(input_test)

fig = plt.figure(facecolor='white', figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(label_test, label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()